In [1]:
import os

In [2]:
os.environ['SPOTIPY_CLIENT_ID']='f30051ca497649079b904ef7476af7d2'
os.environ['SPOTIPY_CLIENT_SECRET']='57693afba78047d8a1b658bca978f129'
os.environ['SPOTIPY_REDIRECT_URI']='https://www.jvm16.xyz/#/english'

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [20]:
import urllib.request

def download_preview(song_title, path):
    extension = '.mp3'
    
    if extension not in path:
        path = f'{path}{extension}'
    track = sp.search(song_title, limit=1, offset=0, type='track', market=None)
    
    tracks = track['tracks']

    # print(tracks['items'])
    
    preview_url = [item['preview_url'] for item in tracks['items'] if item['preview_url'] is not None]

    if len(preview_url) > 0:
        preview_url = preview_url[0]
    else:        
        print('no preview url')
        preview_url = None
        
    if preview_url is not None:
        urllib.request.urlretrieve(preview_url, path)

        return path
    else:
        return None

    

In [5]:
#download_preview('cayetano carolina durante', './cd_cayetano.mp3')

In [6]:
#download_preview('el himno titular carolina durante', './cd_himno.mp3')

In [7]:
#download_preview('got you amyl and the sniffers', './ats_got_you.mp3')

In [8]:
#download_preview('hertz amyl and the sniffers', './ats_hertz.mp3')

In [9]:
# download_preview('big mouth strikes again the smiths', './ts_big_mouth.mp3')

'./ts_big_mouth.mp3'

In [10]:
# download_preview('there is a light that never goes out the smiths', './ts_light.mp3')

'./ts_light.mp3'

In [11]:
# download_preview('bogus operandi the hives', './th_bogus.mp3')

'./th_bogus.mp3'

In [12]:
# download_preview('come on the hives', './th_comon.mp3')

'./th_comon.mp3'

In [21]:
# download_preview('thunder island jay ferguson', './jf_thunder.mp3')

'./jf_thunder.mp3'

In [14]:
#!pip3.10 install torch torchaudio

  Using cached torch-2.3.1-cp310-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached torchaudio-2.3.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.4 kB)
Using cached torch-2.3.1-cp310-none-macosx_11_0_arm64.whl (61.0 MB)
Using cached torchaudio-2.3.1-cp310-cp310-macosx_11_0_arm64.whl (1.8 MB)


In [11]:
#!pip install transformers datasets librosa soundfile

In [22]:
import torch

torch.set_default_device("cpu")

cpu_device = torch.device("cpu")

In [23]:
# Step 2: Import necessary libraries
import librosa
import numpy as np
from transformers import ClapModel, ClapProcessor

# Step 3: Load your MP3 file and convert it to a waveform array
def load_audio_file(file_path, target_sr=48000):
    # Load the audio file with librosa
    waveform, sr = librosa.load(file_path, sr=None)
    # Resample to target sample rate if needed
    if sr != target_sr:
        waveform = librosa.resample(waveform, orig_sr=sr, target_sr=target_sr)
    return waveform, target_sr


def get_preview_embeddings(audio_file_path):
    # Load the audio file
    audio_waveform, sample_rate = load_audio_file(audio_file_path)
    
    # Ensure the waveform is a numpy array
    audio_waveform = np.array(audio_waveform)
    
    # Step 4: Load the model and processor
    model = ClapModel.from_pretrained("laion/larger_clap_music").to(cpu_device)
    processor = ClapProcessor.from_pretrained("laion/larger_clap_music")
    
    # Step 5: Process the waveform with the ClapProcessor
    inputs = processor(audios=audio_waveform, sampling_rate=sample_rate, return_tensors="pt").to(cpu_device)
    
    # Step 6: Get audio features using the model
    audio_embed_my_heart_will_go_on = model.get_audio_features(**inputs)

    return audio_embed_my_heart_will_go_on


/Users/jose/.pyenv/versions/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
audio_embed_my_heart_will_go_on = get_preview_embeddings('./test.mp3').detach().numpy()

In [25]:
audio_embed_cayetano = get_preview_embeddings('./cd_cayetano.mp3').detach().numpy()

In [26]:
audio_embed_himno = get_preview_embeddings('./cd_himno.mp3').detach().numpy()

In [27]:
audio_embed_gotyou = get_preview_embeddings('./ats_got_you.mp3').detach().numpy()

In [85]:
audio_embed_hertz = get_preview_embeddings('./ats_hertz.mp3').detach().numpy()

In [28]:
audio_embed_ts_big_mouth = get_preview_embeddings('./ts_big_mouth.mp3').detach().numpy()

In [29]:
audio_embed_ts_light = get_preview_embeddings('./ts_light.mp3').detach().numpy()

In [30]:
audio_embed_th_bogus = get_preview_embeddings('./th_bogus.mp3').detach().numpy()

In [31]:
audio_embed_th_comon = get_preview_embeddings('./th_comon.mp3').detach().numpy()

In [32]:
audio_embed_jf_thunder = get_preview_embeddings('./jf_thunder.mp3').detach().numpy()

In [33]:
from scipy.spatial.distance import cosine, euclidean, cityblock, chebyshev, mahalanobis

In [34]:
# Function to compute distances between two embeddings
def compute_distances(embedding1, embedding2):
    cos_dist = cosine(embedding1, embedding2)
    euclid_dist = euclidean(embedding1, embedding2)
    return cos_dist, euclid_dist


# Compute various distances between two embeddings
def compute_distances(embedding1, embedding2, inv_cov_matrix=None):
    # Cosine Similarity
    cos_dist = cosine(embedding1, embedding2)
    
    # Euclidean Distance
    euclid_dist = euclidean(embedding1, embedding2)
    
    # Manhattan Distance
    manhattan_dist = cityblock(embedding1, embedding2)
    
    # Chebyshev Distance
    chebyshev_dist = chebyshev(embedding1, embedding2)

    try:
        # Compute inverse of covariance matrix for Mahalanobis distance (if needed)
        # Assuming you have multiple embeddings to calculate the covariance matrix
        embeddings = np.array([embedding1, embedding2])  # Replace with your actual embeddings
        cov_matrix = np.cov(embeddings, rowvar=False)
        inv_cov_matrix = np.linalg.inv(cov_matrix)    
    except:
        inv_cov_matrix = None
    
    # Mahalanobis Distance (requires inverse of the covariance matrix)
    mahalanobis_dist = None
    if inv_cov_matrix is not None:
        mahalanobis_dist = mahalanobis(embedding1, embedding2, inv_cov_matrix)
    
    return {
        "cosine_distance": cos_dist,
        "euclidean_distance": euclid_dist,
        "manhattan_distance": manhattan_dist,
        "chebyshev_distance": chebyshev_dist,
        "mahalanobis_distance": mahalanobis_dist
    }


In [ ]:
sp.search()

In [35]:
def get_audio_features(sp, search_string):
    """
    Retrieves audio features for a given track search string using Spotipy.

    Parameters:
    sp (spotipy.Spotify): An authenticated Spotipy client instance.
    search_string (str): The search string for the track (e.g., "Shape of You Ed Sheeran").

    Returns:
    dict: A dictionary of audio features for the track.
    """
    # Search for the track
    results = sp.search(q=search_string, type='track')
    if not results['tracks']['items']:
        return None

    # Extract the track ID
    track_id = results['tracks']['items'][0]['id']

    # Get audio features for the track
    audio_features = sp.audio_features(track_id)[0]

    # Prepare the dictionary of audio features
    features = {
        "danceability": audio_features['danceability'],
        "energy": audio_features['energy'],
        "key": audio_features['key'],
        "loudness": audio_features['loudness'],
        "mode": audio_features['mode'],
        "speechiness": audio_features['speechiness'],
        "acousticness": audio_features['acousticness'],
        "instrumentalness": audio_features['instrumentalness'],
        "liveness": audio_features['liveness'],
        "valence": audio_features['valence'],
        "tempo": audio_features['tempo']
    }

    return features

In [36]:
cayetano_feats = get_audio_features(sp, 'cayetano carolina durante')

In [37]:
himno_feats = get_audio_features(sp, 'el himno titular carolina durante')

In [38]:
gotyou_feats = get_audio_features(sp, 'got you amyl and the sniffers')

In [39]:
hertz_feats = get_audio_features(sp, 'hertz amyl and the sniffers')

In [40]:
heart_feats = get_audio_features(sp, 'My Heart Will Go On')

In [41]:
mouth_feats = get_audio_features(sp, 'big mouth strikes again the smiths')

In [42]:
light_feats = get_audio_features(sp, 'there is a light that never goes out the smiths')

In [43]:
bogus_feats = get_audio_features(sp, 'bogus operandi the hives')

In [44]:
comon_feats = get_audio_features(sp, 'come on the hives')

In [45]:
thunder_feats = get_audio_features(sp, 'thunder island jay ferguson')

In [46]:
import pandas as pd

feats_df = pd.DataFrame([cayetano_feats,
                         himno_feats,
                         hertz_feats,
                         gotyou_feats,
                         heart_feats,
                         mouth_feats,
                         light_feats,
                         bogus_feats,
                         comon_feats,
                         thunder_feats
                        ])

In [47]:
for c in feats_df.columns:
    feats_df[c] = ((feats_df[c] - feats_df[c].min()) 
                   /  
                   max(feats_df[c].max() - feats_df[c].min(), 0.00001)
                  ) 

In [48]:
feats_df['song'] = ['cayetano', 'himno', 'hertz', 'gotyou', 'heart', 'mouth', 'light', 'bogus', 'comon', 'thunder']

In [49]:
feats_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,song
0,0.612613,0.990251,0.363636,0.836765,1.0,0.335231,0.027871,0.000000,0.515351,0.684889,0.449389,cayetano
1,0.201201,0.987465,0.363636,0.787016,1.0,0.256799,0.187270,0.000000,0.576754,0.585820,0.020690,himno
2,0.000000,0.887187,0.090909,0.896956,0.0,0.487666,0.000821,0.000246,0.171053,0.887801,1.000000,hertz
3,1.000000,0.668524,0.090909,0.732736,1.0,0.169513,0.002545,0.562706,0.059211,0.706374,0.014822,gotyou
4,0.297297,0.000000,0.363636,0.000000,1.0,0.027198,1.000000,0.000088,0.000000,0.000000,0.071318,heart
5,0.216216,0.870474,0.090909,0.661807,0.0,0.089184,0.094777,0.000000,0.923246,0.955837,0.458212,mouth
6,0.582583,0.686630,0.363636,0.578662,1.0,0.000000,0.053183,0.000000,0.041667,1.000000,0.473024,light
7,0.078078,1.000000,1.000000,1.000000,0.0,1.000000,0.028965,0.000513,0.442982,0.052877,0.568595,bogus
8,0.612613,0.793872,0.818182,0.690474,1.0,0.160025,0.008948,0.214521,1.000000,0.812604,0.000000,comon
9,0.504505,0.615599,0.000000,0.392572,1.0,0.003795,0.000000,1.000000,0.936404,0.899737,0.382048,thunder


In [50]:
n_feats = len(feats_df[feats_df['song']=='cayetano'].drop(columns=['song', 'mode', 'key']).values.squeeze())

In [86]:
embed_df = pd.DataFrame([audio_embed_my_heart_will_go_on.flatten(),
                         audio_embed_cayetano.flatten(),
                         audio_embed_himno.flatten(),
                         audio_embed_gotyou.flatten(),
                         audio_embed_hertz.flatten(),
                         audio_embed_ts_big_mouth.flatten(),
                         audio_embed_ts_light.flatten(),
                         audio_embed_th_bogus.flatten(),
                         audio_embed_th_comon.flatten(),
                         audio_embed_jf_thunder.flatten()
                        ]
                       )

In [87]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Assuming 'embeddings' is your 2D array of shape (num_samples, 512) containing embeddings
# Replace this with your actual embeddings data

# Perform PCA
pca = PCA(n_components=n_feats)

pca.fit(embed_df)

# Perform t-SNE
tsne = TSNE(n_components=n_feats, perplexity=8.9, method='exact', metric='cosine')

tsne.fit(embed_df)


TSNE(method='exact', metric='cosine', n_components=9, perplexity=8.9)

## Cayetano vs My Heart

In [56]:
compute_distances(audio_embed_cayetano.squeeze(), audio_embed_my_heart_will_go_on.squeeze())

{'cosine_distance': 0.27320782701922364,
 'euclidean_distance': 0.7391995191574097,
 'manhattan_distance': 13.457222,
 'chebyshev_distance': 0.096170545,
 'mahalanobis_distance': None}

In [112]:
compute_distances(feats_df[feats_df['song']=='cayetano'].drop(columns=['song', 'mode', 'key']).values.squeeze(), 
                  feats_df[feats_df['song']=='heart'].drop(columns=['song', 'mode', 'key']).values.squeeze())


{'cosine_distance': 0.891540246454024,
 'euclidean_distance': 2.2146352929226114,
 'manhattan_distance': 5.938988837039051,
 'chebyshev_distance': 1.0,
 'mahalanobis_distance': None}

In [88]:
compute_distances(
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_cayetano.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='cayetano'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )                                
                  , 
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_my_heart_will_go_on.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='heart'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )
)


{'cosine_distance': 0.9256960803047407,
 'euclidean_distance': 2.106345402952978,
 'manhattan_distance': 6.78280534430086,
 'chebyshev_distance': 0.9902506963788301,
 'mahalanobis_distance': 2.088328867879624}

## Cayetano vs Himno

In [57]:
compute_distances(audio_embed_cayetano.squeeze(), audio_embed_himno.squeeze())

{'cosine_distance': 0.17769837379455566,
 'euclidean_distance': 0.5961517691612244,
 'manhattan_distance': 10.686256,
 'chebyshev_distance': 0.08349282,
 'mahalanobis_distance': None}

In [113]:
compute_distances(feats_df[feats_df['song']=='cayetano'].drop(columns=['song', 'mode', 'key']).values.squeeze(), 
                  feats_df[feats_df['song']=='himno'].drop(columns=['song', 'mode', 'key']).values.squeeze())


{'cosine_distance': 0.051878607174507874,
 'euclidean_distance': 0.6632846128226539,
 'manhattan_distance': 1.452749624354583,
 'chebyshev_distance': 0.43514852031193113,
 'mahalanobis_distance': None}

In [89]:
compute_distances(
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_cayetano.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='cayetano'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )                                
                  , 
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_himno.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='himno'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )
)


{'cosine_distance': 0.12649977225948672,
 'euclidean_distance': 0.8971210922632251,
 'manhattan_distance': 2.601384274665579,
 'chebyshev_distance': 0.49127916246652603,
 'mahalanobis_distance': None}

## Got You vs My Heart

In [58]:
compute_distances(audio_embed_gotyou.squeeze(), audio_embed_my_heart_will_go_on.squeeze())

{'cosine_distance': 0.22044043419119974,
 'euclidean_distance': 0.6639887690544128,
 'manhattan_distance': 12.1134405,
 'chebyshev_distance': 0.08746224,
 'mahalanobis_distance': None}

In [114]:
compute_distances(feats_df[feats_df['song']=='gotyou'].drop(columns=['song', 'mode', 'key']).values.squeeze(), 
                  feats_df[feats_df['song']=='heart'].drop(columns=['song', 'mode', 'key']).values.squeeze())

/Users/jose/.pyenv/versions/3.10.13/lib/python3.10/site-packages/scipy/spatial/distance.py:1035: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(m)


{'cosine_distance': 0.8542979857661945,
 'euclidean_distance': 2.08699431425592,
 'manhattan_distance': 5.457557294547486,
 'chebyshev_distance': 0.9998436950146627,
 'mahalanobis_distance': nan}

In [90]:
compute_distances(
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_gotyou.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='gotyou'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )                                
                  , 
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_my_heart_will_go_on.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='heart'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )
)


{'cosine_distance': 0.840159611422218,
 'euclidean_distance': 1.9057262021485988,
 'manhattan_distance': 5.952825884147663,
 'chebyshev_distance': 0.9974550877721072,
 'mahalanobis_distance': 16.131536219658486}

## Got You vs Hertz

In [59]:
compute_distances(audio_embed_gotyou.squeeze(), audio_embed_hertz.squeeze())

{'cosine_distance': 0.1252791210842641,
 'euclidean_distance': 0.5005578398704529,
 'manhattan_distance': 9.066539,
 'chebyshev_distance': 0.07366593,
 'mahalanobis_distance': None}

In [115]:
compute_distances(feats_df[feats_df['song']=='gotyou'].drop(columns=['song', 'mode', 'key']).values.squeeze(), 
                  feats_df[feats_df['song']=='hertz'].drop(columns=['song', 'mode', 'key']).values.squeeze())

{'cosine_distance': 0.4115786052340077,
 'euclidean_distance': 1.9073972422235734,
 'manhattan_distance': 4.494396833342899,
 'chebyshev_distance': 1.0,
 'mahalanobis_distance': None}

In [91]:
compute_distances(
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_gotyou.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='gotyou'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )                                
                  , 
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_hertz.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='hertz'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )
)


{'cosine_distance': 0.422472640224937,
 'euclidean_distance': 1.7118837457680272,
 'manhattan_distance': 5.124980962241088,
 'chebyshev_distance': 1.0,
 'mahalanobis_distance': None}

## Got You vs Cayetano

In [60]:
compute_distances(audio_embed_gotyou.squeeze(), audio_embed_cayetano.squeeze())

{'cosine_distance': 0.1805654871800818,
 'euclidean_distance': 0.6009416580200195,
 'manhattan_distance': 10.601484,
 'chebyshev_distance': 0.07923092,
 'mahalanobis_distance': None}

In [116]:
compute_distances(feats_df[feats_df['song']=='cayetano'].drop(columns=['song', 'mode', 'key']).values.squeeze(), 
                  feats_df[feats_df['song']=='gotyou'].drop(columns=['song', 'mode', 'key']).values.squeeze())

/Users/jose/.pyenv/versions/3.10.13/lib/python3.10/site-packages/scipy/spatial/distance.py:1035: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(m)


{'cosine_distance': 0.2716057525677148,
 'euclidean_distance': 1.4899267691341027,
 'manhattan_distance': 3.4696783856550395,
 'chebyshev_distance': 1.0,
 'mahalanobis_distance': nan}

In [92]:
compute_distances(
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_gotyou.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='gotyou'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )                                
                  , 
                  np.concatenate(
                                  (
                                  pca.transform(audio_embed_cayetano.reshape(1, -1)).squeeze(),
                                  feats_df[feats_df['song']=='cayetano'].drop(columns=['song', 'mode', 'key']).values.squeeze()
                                  )
                                )
)


/Users/jose/.pyenv/versions/3.10.13/lib/python3.10/site-packages/scipy/spatial/distance.py:1035: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(m)


{'cosine_distance': 0.2160222815520063,
 'euclidean_distance': 1.1730321961379908,
 'manhattan_distance': 3.84649884041925,
 'chebyshev_distance': 0.5627062706270627,
 'mahalanobis_distance': nan}

In [ ]:
# TODO: al obtener la cosine distance con features vs pca embedding + features la diferencia de distancias entre ambos approaches es muy similar
# se puede seguir adelante sólo con distancias de features para una v1. Además tiene una ventaja muy importante: no se necesitan las samples
# que algunas canciones no las tienen. También, ocupan menos espacio en disco.

In [123]:
pl_tracks = sp.playlist_tracks("https://open.spotify.com/playlist/5ZzVrO1Y9plnOCoSmfW4Hk?si=e826324136a64261")

In [131]:
pl_tracks

{'href': 'https://api.spotify.com/v1/playlists/5ZzVrO1Y9plnOCoSmfW4Hk/tracks?offset=0&limit=100&additional_types=track',
 'items': [{'added_at': '2023-12-24T18:24:04Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/1199533427'},
    'href': 'https://api.spotify.com/v1/users/1199533427',
    'id': '1199533427',
    'type': 'user',
    'uri': 'spotify:user:1199533427'},
   'is_local': False,
   'primary_color': None,
   'track': {'preview_url': 'https://p.scdn.co/mp3-preview/76c2bea145113e9ca00f95ae7eec9950893c1cdf?cid=f30051ca497649079b904ef7476af7d2',
    'available_markets': ['AR',
     'AU',
     'AT',
     'BE',
     'BO',
     'BR',
     'BG',
     'CA',
     'CL',
     'CO',
     'CR',
     'CY',
     'CZ',
     'DK',
     'DO',
     'DE',
     'EC',
     'EE',
     'SV',
     'FI',
     'FR',
     'GR',
     'GT',
     'HN',
     'HK',
     'HU',
     'IS',
     'IE',
     'IT',
     'LV',
     'LT',
     'LU',
     'MY',
     'MT',
     'MX',
     '

## Idea for now:

- Quedarnos con canciones que enganchen con alguna canción referencia por tener distancia de features (cosine) < 0.5
- Las que son de bandas que tenemos como referencia o en favoritos (ver yaml), nos las quedamos
- Guardar en una tabla con qué canción se está enganchando, las features de una, de otra, y la distancia

- Extra: un proceso que haga lo siguiente: por cada canción añadida a playlist curated de la anterior ejecución, mirar si se ha añadido a 
favoritos. Se va guardando una tabla con todo esto, siendo 1 si añadido a favoritos y 0 en caso contrario. Se hace una tabla con las features,
distancia coseno a los top 5 canciones referencia, y si es posible los embeddings, junto con la distancia coseno a los top 5 canciones referencia.

La idea es crear un modelo que decida si una canción es de mi agrado o no, con un poco más inteligencia que un filtro simple.